# Create test data sample with 33k rows per sentiment

In [1]:
import pandas as pd
import time

# Load data from CSV
df = pd.read_csv('rt_data.csv')  # Adjust path accordingly

# Desired sample size of each category for top critics and non-top critics
sample_size = 33000

# Sample from top critics
df_top_critics = df[df['top_critic'] == True].sample(
    n=min(len(df[df['top_critic'] == True]), sample_size), random_state=42)

# Sample from non-top critics
df_not_top_critics = df[df['top_critic'] == False].sample(
    n=min(len(df[df['top_critic'] == False]), sample_size), random_state=42)

# Combine the samples
df_balanced = pd.concat([df_top_critics, df_not_top_critics])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset to a new CSV file
timestr = time.strftime("%Y%m%d-%H%M%S")
df_balanced.to_csv(f'balanced_rt_reviews_{timestr}.csv', index=False)


C:\Users\erwin\AppData\Local\Temp\ipykernel_19584\2556771392.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('rt_data.csv')  # Adjust path accordingly


# logistic regression

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:

from sklearn.linear_model import LogisticRegression

def getAndTrainModel(X_train, y_train):
    m = LogisticRegression(solver='saga', multi_class='auto', max_iter=2000, class_weight='balanced')
    m.fit(X_train, y_train)
    return m 

In [4]:
# Loading data from a CSV file
df = pd.read_csv('balanced_rt_reviews_20240207-193333.csv')  # Adjust the filename to your actual file path

# Count the number of NaN or empty strings in 'review_detail' for the entire dataset
missing_or_empty_count = df['review_detail'].isna().sum() + (df['review_detail'] == '').sum()

# Print the count
print("Total missing or empty 'review_detail' values in the dataset:", missing_or_empty_count)

df = df.dropna(subset=['review_detail'])

Total missing or empty 'review_detail' values in the dataset: 7


In [5]:
texts = df['review_detail'].values
# If 'top_critic' is boolean, you can directly use it as an integer target variable
top_critics = df['top_critic'].astype(int).values  # Converts boolean to 0 (False) and 1 (True)

# Vectorizing text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, top_critics, test_size=0.2, random_state=42)

# Ensure the test set is not too small
if X_test.shape[0] < 1:
    raise ValueError("Test set is too small. Consider reducing the test_size parameter or adding more data.")

# Training a model with adjusted parameters
model = getAndTrainModel(X_train=X_train, y_train=y_train)

In [7]:
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
import numpy as np

# Predicting
predictions = model.predict(X_test)

# Evaluating with accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Calculate and print MAE
ra = roc_auc_score(y_test, predictions)
print("ROC AUC score:", ra)


# Generating and printing the classification report
report = classification_report(y_test, predictions, target_names=["False","True"])
print("Classification Report:")
print(report)

Accuracy: 0.5849685582241079
ROC AUC score: 0.5851587288466016
Classification Report:
              precision    recall  f1-score   support

       False       0.59      0.56      0.58      6660
        True       0.58      0.61      0.59      6539

    accuracy                           0.58     13199
   macro avg       0.59      0.59      0.58     13199
weighted avg       0.59      0.58      0.58     13199

